In [7]:
# ============================================
#  Sistema de Gestión - Empresa de Transporte
#  POO: Abstracción, Herencia, Encapsulamiento, Polimorfismo
#  HECHO POR: Juan Diego Trujillo Narvaez
# ============================================

from __future__ import annotations
from abc import ABC, abstractmethod
from dataclasses import dataclass
from enum import Enum, auto
from typing import Optional, List, Dict
import itertools
import datetime as dt

# Utilidades y ayudantes
_id_counter = itertools.count(1)

def _new_id(prefix: str) -> str:
    return f"{prefix}-{next(_id_counter):04d}"

class VehicleStatus(Enum):
    DISPONIBLE = auto()
    EN_VIAJE = auto()
    EN_TALLER = auto()
    FUERA_DE_SERVICIO = auto()

def banner(title: str) -> str:
    line = "═" * (len(title) + 2)
    return f"\n{line}\n {title}\n{line}"

def input_int(msg: str, minimo: Optional[int]=None) -> int:
    while True:
        try:
            v = int(input(msg).strip())
            if minimo is not None and v < minimo:
                print(f"Debe ser >= {minimo}.")
                continue
            return v
        except ValueError:
            print("Valor inválido. Intenta de nuevo.")

def input_float(msg: str, minimo: Optional[float]=None) -> float:
    while True:
        try:
            v = float(input(msg).strip())
            if minimo is not None and v < minimo:
                print(f"Debe ser >= {minimo}.")
                continue
            return v
        except ValueError:
            print("Valor inválido. Intenta de nuevo.")

def press_enter():
    input("\nPresiona ENTER para continuar...")


# Historial y Entidades
@dataclass
class TripLog:
    fecha: dt.datetime
    conductor_id: str
    vehiculo_id: str
    km: float
    comentario: str = ""

@dataclass
class MaintenanceRecord:
    fecha: dt.datetime
    mecanico_id: str
    vehiculo_id: str
    detalle: str
    costo: float = 0.0


# Abstracción: Empleado
class Employee(ABC):
    """Clase abstracta para representar empleados."""
    def __init__(self, nombre: str):
        self._id: str = _new_id("EMP")   # protegido
        self.__nombre: str = nombre      # privado
        self.__activo: bool = True

    @property
    def id(self) -> str:
        return self._id

    @property
    def nombre(self) -> str:
        return self.__nombre

    @nombre.setter
    def nombre(self, value: str):
        if not value.strip():
            raise ValueError("El nombre no puede estar vacío.")
        self.__nombre = value

    @property
    def activo(self) -> bool:
        return self.__activo

    def desactivar(self):
        self.__activo = False

    def activar(self):
        self.__activo = True

    @abstractmethod
    def rol(self) -> str: ...

class Driver(Employee):
    def __init__(self, nombre: str, licencia: str):
        super().__init__(nombre)
        self.__licencia: str = licencia
        self.__vehiculo_asignado: Optional[str] = None

    @property
    def licencia(self) -> str:
        return self.__licencia

    @property
    def vehiculo_asignado(self) -> Optional[str]:
        return self.__vehiculo_asignado

    def asignar_vehiculo(self, vehiculo_id: Optional[str]):
        self.__vehiculo_asignado = vehiculo_id

    def rol(self) -> str:
        return "Conductor"

class Mechanic(Employee):
    def __init__(self, nombre: str, especialidad: str):
        super().__init__(nombre)
        self.__especialidad: str = especialidad

    @property
    def especialidad(self) -> str:
        return self.__especialidad

    def rol(self) -> str:
        return "Mecánico"

# Abstracción: Vehículo
class Vehicle(ABC):
    def __init__(self, marca: str, modelo: str, anio: int, capacidad: int):
        self._id: str = _new_id("VEH")
        self.__marca: str = marca
        self.__modelo: str = modelo
        self.__anio: int = anio
        self.__capacidad: int = capacidad
        self.__odometro_km: float = 0.0
        self.__status: VehicleStatus = VehicleStatus.DISPONIBLE
        self.__ultimo_mto_km: float = 0.0

    @property
    def id(self) -> str: return self._id
    @property
    def marca(self) -> str: return self.__marca
    @property
    def modelo(self) -> str: return self.__modelo
    @property
    def anio(self) -> int: return self.__anio
    @property
    def capacidad(self) -> int: return self.__capacidad
    @property
    def odometro_km(self) -> float: return self.__odometro_km
    @property
    def status(self) -> VehicleStatus: return self.__status

    def _set_status(self, status: VehicleStatus):
        self.__status = status

    def registrar_viaje(self, km: float):
        if km <= 0:
            raise ValueError("Los km del viaje deben ser positivos.")
        if self.__status != VehicleStatus.DISPONIBLE:
            raise RuntimeError("El vehículo no está disponible para viajar.")
        self._set_status(VehicleStatus.EN_VIAJE)
        self.__odometro_km += km
        self._set_status(VehicleStatus.DISPONIBLE)

    def requiere_mantenimiento(self, umbral_km: float = 10000.0) -> bool:
        return (self.__odometro_km - self.__ultimo_mto_km) >= umbral_km

    def marcar_mantenimiento_realizado(self):
        self.__ultimo_mto_km = self.__odometro_km

    @abstractmethod
    def costo_por_km(self) -> float: ...
    @abstractmethod
    def arrancar(self) -> str: ...

    def describir(self) -> str:
        base = f"{self.__class__.__name__}({self.marca} {self.modelo} {self.anio})"
        return f"{base} - Capacidad: {self.capacidad} - Odo: {self.odometro_km:.1f} km - Estado: {self.status.name}"

class Car(Vehicle):
    def costo_por_km(self) -> float: return 0.45
    def arrancar(self) -> str: return "El auto encendió: Listo para salir."

class Motorcycle(Vehicle):
    def costo_por_km(self) -> float: return 0.25
    def arrancar(self) -> str: return "La moto prendió: ¡Fuimonos!"

class Truck(Vehicle):
    def __init__(self, marca: str, modelo: str, anio: int, capacidad: int, ejes: int):
        super().__init__(marca, modelo, anio, capacidad)
        self.__ejes = ejes

    @property
    def ejes(self) -> int: return self.__ejes
    def costo_por_km(self) -> float: return 1.20
    def arrancar(self) -> str: return "El camión pesado ruge: Potencia lista."
    def describir(self) -> str:
        return f"{super().describir()} - Ejes: {self.ejes}"


# Núcleo del Sistema
class Company:
    def __init__(self, nombre: str):
        self.nombre = nombre
        self.vehiculos: Dict[str, Vehicle] = {}
        self.empleados: Dict[str, Employee] = {}
        self.trips: List[TripLog] = []
        self.maintenances: List[MaintenanceRecord] = []

    def add_vehicle(self, v: Vehicle) -> str:
        self.vehiculos[v.id] = v
        return v.id

    def add_employee(self, e: Employee) -> str:
        self.empleados[e.id] = e
        return e.id

    # CRUD de Vehículos
    def crear_vehiculo(self, tipo: str, marca: str, modelo: str, anio: int, capacidad: int,
                       ejes: Optional[int] = None) -> str:
        t = (tipo or "").strip().lower()
        if t == "car":
            v = Car(marca, modelo, anio, capacidad)
        elif t == "motorcycle":
            v = Motorcycle(marca, modelo, anio, capacidad)
        elif t == "truck":
            if ejes is None or ejes <= 0:
                raise ValueError("Para 'truck' debes especificar un número de ejes > 0.")
            v = Truck(marca, modelo, anio, capacidad, ejes=ejes)
        else:
            raise ValueError("Tipo de vehículo no válido. Usa: 'car', 'motorcycle' o 'truck'.")
        self.vehiculos[v.id] = v
        print(f"✅ Vehículo creado: {v.describir()}")
        return v.id

    def eliminar_vehiculo(self, vehiculo_id: str) -> None:
        v = self._get_vehicle(vehiculo_id)
        if v.status == VehicleStatus.EN_VIAJE:
            raise RuntimeError("No se puede eliminar un vehículo que está EN VIAJE.")
        asignados = [e for e in self.empleados.values()
                     if isinstance(e, Driver) and e.vehiculo_asignado == vehiculo_id]
        if asignados:
            for d in asignados:
                d.asignar_vehiculo(None)
            print(f"ℹ️  {len(asignados)} conductor(es) desasignados de {vehiculo_id} antes de eliminarlo.")
        del self.vehiculos[vehiculo_id]
        print(f"🗑️  Vehículo {vehiculo_id} eliminado del inventario.")

    def listar_vehiculos(self, status: Optional[VehicleStatus] = None) -> List[str]:
        items: List[str] = []
        for v in self.vehiculos.values():
            if status is None or v.status == status:
                items.append(v.describir())
        return items
    # Fin CRUD de Vehículos

    # Asignaciones
    def asignar_conductor(self, driver_id: str, vehiculo_id: Optional[str]):
        driver = self._get_driver(driver_id)
        if vehiculo_id is not None:
            vehiculo = self._get_vehicle(vehiculo_id)
            if vehiculo.status != VehicleStatus.DISPONIBLE:
                raise RuntimeError("No se puede asignar un vehículo no disponible.")
            driver.asignar_vehiculo(vehiculo.id)
        else:
            driver.asignar_vehiculo(None)

    # Operaciones
    def iniciar_viaje(self, driver_id: str, km: float, comentario: str = ""):
        driver = self._get_driver(driver_id)
        if not driver.activo:
            raise RuntimeError("Conductor inactivo.")
        if driver.vehiculo_asignado is None:
            raise RuntimeError("El conductor no tiene vehículo asignado.")
        vehiculo = self._get_vehicle(driver.vehiculo_asignado)
        mensaje_arranque = vehiculo.arrancar()
        vehiculo.registrar_viaje(km)
        self.trips.append(TripLog(
            fecha=dt.datetime.now(),
            conductor_id=driver.id,
            vehiculo_id=vehiculo.id,
            km=km,
            comentario=comentario
        ))
        costo = km * vehiculo.costo_por_km()
        print(f"{mensaje_arranque}\nViaje registrado: {km:.1f} km | Costo estimado: ${costo:,.2f}")
        if vehiculo.requiere_mantenimiento():
            print("⚠️  Aviso: Este vehículo alcanzó el limite de mantenimiento.")

    def enviar_a_taller(self, vehiculo_id: str):
        v = self._get_vehicle(vehiculo_id)
        if v.status == VehicleStatus.EN_VIAJE:
            raise RuntimeError("No se puede enviar a taller durante un viaje.")
        v._set_status(VehicleStatus.EN_TALLER)

    def registrar_mantenimiento(self, mechanic_id: str, vehiculo_id: str, detalle: str, costo: float = 0.0):
        mech = self._get_mechanic(mechanic_id)
        v = self._get_vehicle(vehiculo_id)
        if v.status != VehicleStatus.EN_TALLER:
            raise RuntimeError("El vehículo debe estar en TALLER para registrar mantenimiento.")
        self.maintenances.append(MaintenanceRecord(
            fecha=dt.datetime.now(),
            mecanico_id=mech.id,
            vehiculo_id=v.id,
            detalle=detalle,
            costo=costo
        ))
        v.marcar_mantenimiento_realizado()
        v._set_status(VehicleStatus.DISPONIBLE)
        print(f"🛠️  Mantenimiento registrado por {mech.nombre} ({mech.especialidad}) -> {detalle}. Costo: ${costo:,.2f}")

    # Reportes
    def resumen_flota(self) -> str:
        lines = [banner(f"Estado actual de la flota - {self.nombre}")]
        for v in self.vehiculos.values():
            lines.append(" - " + v.describir())
        return "\n".join(lines)

    resumen_flotilla = resumen_flota

    def resumen_personal(self) -> str:
        lines = [banner("Personal")]
        for e in self.empleados.values():
            info = f"{e.rol():<10} | {e.nombre:<20} | ID: {e.id} | Activo: {'Sí' if e.activo else 'No'}"
            if isinstance(e, Driver):
                info += f" | Vehículo asignado: {e.vehiculo_asignado or 'Ninguno'}"
            if isinstance(e, Mechanic):
                info += f" | Especialidad: {e.especialidad}"
            lines.append(" - " + info)
        return "\n".join(lines)

    def resumen_operaciones(self) -> str:
        lines = [banner("Historial de viajes")]
        if not self.trips:
            lines.append(" (sin viajes aún)")
        else:
            for t in self.trips[-10:]:
                lines.append(f" - {t.fecha:%Y-%m-%d %H:%M} | Veh:{t.vehiculo_id} | Con:{t.conductor_id} | {t.km:.1f} km | {t.comentario}")
        lines.append(banner("Historial de mantenimientos"))
        if not self.maintenances:
            lines.append(" (sin mantenimientos aún)")
        else:
            for m in self.maintenances[-10:]:
                lines.append(f" - {m.fecha:%Y-%m-%d %H:%M} | Veh:{m.vehiculo_id} | Mec:{m.mecanico_id} | {m.detalle} | ${m.costo:,.2f}")
        return "\n".join(lines)

    # Internos
    def _get_vehicle(self, vehiculo_id: str) -> Vehicle:
        v = self.vehiculos.get(vehiculo_id)
        if not v:
            raise KeyError("Vehículo no encontrado.")
        return v

    def _get_driver(self, driver_id: str) -> Driver:
        e = self.empleados.get(driver_id)
        if not isinstance(e, Driver):
            raise KeyError("Conductor no encontrado.")
        return e

    def _get_mechanic(self, mechanic_id: str) -> Mechanic:
        e = self.empleados.get(mechanic_id)
        if not isinstance(e, Mechanic):
            raise KeyError("Mecánico no encontrado.")
        return e


# SEED de datos para probar
def seed_company() -> Company:
    empresa = Company("TransLogix S.A.S.")
    # Vehículos
    v1 = Car("Toyota", "Corolla", 2021, capacidad=5)
    v2 = Motorcycle("Yamaha", "FZ25", 2022, capacidad=2)
    v3 = Truck("Volvo", "FH", 2019, capacidad=20, ejes=3)
    empresa.add_vehicle(v1); empresa.add_vehicle(v2); empresa.add_vehicle(v3)
    # Empleados
    c1 = Driver("Ana Ledesma", licencia="B1")
    c2 = Driver("Carlos Gómez", licencia="C2")
    m1 = Mechanic("Julio Pérez", especialidad="Sistemas de freno")
    m2 = Mechanic("Luisa Ríos", especialidad="Motores diésel")
    empresa.add_employee(c1); empresa.add_employee(c2); empresa.add_employee(m1); empresa.add_employee(m2)
    # Asignaciones iniciales
    empresa.asignar_conductor(c1.id, v1.id)  # Ana -> Auto
    empresa.asignar_conductor(c2.id, v3.id)  # Carlos -> Camión
    return empresa


# Menú de consola
def mostrar_menu():
    print(banner("Menú - Gestión de Empresa de Transporte"))
    print("1) Crear vehículo")
    print("2) Listar vehículos")
    print("3) Eliminar vehículo")
    print("4) Asignar conductor a vehículo / desasignar")
    print("5) Iniciar viaje")
    print("6) Enviar vehículo a taller")
    print("7) Registrar mantenimiento")
    print("8) Ver resumen de flota")
    print("9) Ver resumen de personal")
    print("10) Ver resumen de operaciones")
    print("0) Salir")

def seleccionar_status() -> Optional[VehicleStatus]:
    print("\nFiltro por estado (ENTER para no filtrar):")
    print("1) DISPONIBLE  2) EN_VIAJE  3) EN_TALLER  4) FUERA_DE_SERVICIO")
    op = input("Elige (1-4) o ENTER: ").strip()
    mapa = {"1": VehicleStatus.DISPONIBLE, "2": VehicleStatus.EN_VIAJE,
            "3": VehicleStatus.EN_TALLER, "4": VehicleStatus.FUERA_DE_SERVICIO}
    return mapa.get(op, None)

def opcion_crear_vehiculo(empresa: Company):
    print(banner("Crear vehículo"))
    tipo = input("Tipo [car|motorcycle|truck]: ").strip().lower()
    marca = input("Marca: ").strip()
    modelo = input("Modelo: ").strip()
    anio = input_int("Año: ", minimo=1900)
    capacidad = input_int("Capacidad (personas/ton): ", minimo=1)
    ejes = None
    if tipo == "truck":
        ejes = input_int("Ejes del camión: ", minimo=1)
    try:
        vid = empresa.crear_vehiculo(tipo, marca, modelo, anio, capacidad, ejes=ejes)
        print(f"ID nuevo: {vid}")
    except Exception as ex:
        print("Error:", ex)

def opcion_listar_vehiculos(empresa: Company):
    print(banner("Listar vehículos"))
    status = seleccionar_status()
    lista = empresa.listar_vehiculos(status=status)
    if not lista:
        print("No hay vehículos con ese criterio.")
    else:
        for desc in lista:
            print(" -", desc)

def opcion_eliminar_vehiculo(empresa: Company):
    print(banner("Eliminar vehículo"))
    vid = input("ID del vehículo a eliminar: ").strip()
    try:
        empresa.eliminar_vehiculo(vid)
    except Exception as ex:
        print("Error:", ex)

def opcion_asignar_conductor(empresa: Company):
    print(banner("Asignar/Desasignar conductor"))
    # Mostrar conductores
    drivers = [e for e in empresa.empleados.values() if isinstance(e, Driver)]
    if not drivers:
        print("No hay conductores registrados.")
        return
    for d in drivers:
        print(f" - {d.id} | {d.nombre} | Vehículo asignado: {d.vehiculo_asignado or 'Ninguno'}")
    did = input("ID del conductor: ").strip()
    modo = input("¿Asignar (A) o Desasignar (D)? ").strip().lower()
    try:
        if modo == "a":
            # Mostrar vehículos disponibles
            disp = [v for v in empresa.vehiculos.values() if v.status == VehicleStatus.DISPONIBLE]
            if not disp:
                print("No hay vehículos disponibles.")
                return
            for v in disp:
                print(f" - {v.id} | {v.describir()}")
            vid = input("ID del vehículo a asignar: ").strip()
            empresa.asignar_conductor(did, vid)
            print("Conductor asignado.")
        elif modo == "d":
            empresa.asignar_conductor(did, None)
            print("Conductor desasignado.")
        else:
            print("Opción inválida.")
    except Exception as ex:
        print("Error:", ex)

def opcion_iniciar_viaje(empresa: Company):
    print(banner("Iniciar viaje"))

    # Mostrar conductores con vehículo
    drivers = [e for e in empresa.empleados.values()
               if isinstance(e, Driver) and e.vehiculo_asignado is not None]
    if not drivers:
        print("No hay conductores con vehículo asignado.")
        return
    for d in drivers:
        print(f" - {d.id} | {d.nombre} | Vehículo: {d.vehiculo_asignado}")
    did = input("ID del conductor: ").strip()
    km = input_float("Kilómetros del viaje: ", minimo=0.1)
    comentario = input("Comentario (opcional): ").strip()
    try:
        empresa.iniciar_viaje(did, km, comentario)
    except Exception as ex:
        print("Error:", ex)

def opcion_enviar_taller(empresa: Company):
    print(banner("Enviar vehículo a taller"))
    for v in empresa.vehiculos.values():
        print(f" - {v.id} | {v.describir()}")
    vid = input("ID del vehículo: ").strip()
    try:
        empresa.enviar_a_taller(vid)
        print("Vehículo enviado a taller.")
    except Exception as ex:
        print("Error:", ex)

def opcion_registrar_mantenimiento(empresa: Company):
    print(banner("Registrar mantenimiento"))
    # Mostrar mecánicos
    mechs = [e for e in empresa.empleados.values() if isinstance(e, Mechanic)]
    if not mechs:
        print("No hay mecánicos registrados.")
        return
    for m in mechs:
        print(f" - {m.id} | {m.nombre} | {m.especialidad}")
    mid = input("ID mecánico: ").strip()

    # Mostrar vehículos EN TALLER
    talleres = [v for v in empresa.vehiculos.values() if v.status == VehicleStatus.EN_TALLER]
    if not talleres:
        print("No hay vehículos en taller. Primero envía uno a taller.")
        return
    for v in talleres:
        print(f" - {v.id} | {v.describir()}")
    vid = input("ID del vehículo: ").strip()

    detalle = input("Detalle del mantenimiento: ").strip()
    costo = input_float("Costo (0 si no aplica): ", minimo=0.0)
    try:
        empresa.registrar_mantenimiento(mid, vid, detalle, costo)
    except Exception as ex:
        print("Error:", ex)

def main_menu():
    empresa = seed_company()
    print(banner("Sistema iniciado con datos de prueba"))
    print(empresa.resumen_personal())
    print(empresa.resumen_flota())

    while True:
        mostrar_menu()
        opcion = input("\nElige una opción: ").strip()
        if opcion == "1":
            opcion_crear_vehiculo(empresa); press_enter()
        elif opcion == "2":
            opcion_listar_vehiculos(empresa); press_enter()
        elif opcion == "3":
            opcion_eliminar_vehiculo(empresa); press_enter()
        elif opcion == "4":
            opcion_asignar_conductor(empresa); press_enter()
        elif opcion == "5":
            opcion_iniciar_viaje(empresa); press_enter()
        elif opcion == "6":
            opcion_enviar_taller(empresa); press_enter()
        elif opcion == "7":
            opcion_registrar_mantenimiento(empresa); press_enter()
        elif opcion == "8":
            print(empresa.resumen_flota()); press_enter()
        elif opcion == "9":
            print(empresa.resumen_personal()); press_enter()
        elif opcion == "10":
            print(empresa.resumen_operaciones()); press_enter()
        elif opcion == "0":
            print("\nSaliendo. ¡Gracias por usar el sistema!")
            break
        else:
            print("Opción inválida.")

if __name__ == "__main__":

    main_menu()


══════════════════════════════════════
 Sistema iniciado con datos de prueba
══════════════════════════════════════

══════════
 Personal
══════════
 - Conductor  | Ana Ledesma          | ID: EMP-0004 | Activo: Sí | Vehículo asignado: VEH-0001
 - Conductor  | Carlos Gómez         | ID: EMP-0005 | Activo: Sí | Vehículo asignado: VEH-0003
 - Mecánico   | Julio Pérez          | ID: EMP-0006 | Activo: Sí | Especialidad: Sistemas de freno
 - Mecánico   | Luisa Ríos           | ID: EMP-0007 | Activo: Sí | Especialidad: Motores diésel

═══════════════════════════════════════════════
 Estado actual de la flota - TransLogix S.A.S.
═══════════════════════════════════════════════
 - Car(Toyota Corolla 2021) - Capacidad: 5 - Odo: 0.0 km - Estado: DISPONIBLE
 - Motorcycle(Yamaha FZ25 2022) - Capacidad: 2 - Odo: 0.0 km - Estado: DISPONIBLE
 - Truck(Volvo FH 2019) - Capacidad: 20 - Odo: 0.0 km - Estado: DISPONIBLE - Ejes: 3

═════════════════════════════════════════
 Menú - Gestión de Empresa de Tra